## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## Read cleaned data files

### Read cleaned jobs listings file and save in dataframe

In [2]:
cleaned_jobs_listings_df = pd.read_csv("cleaned_jobs_listings.csv")

### Check columns info in jobs listings dataframe

In [3]:
cleaned_jobs_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3690 entries, 0 to 3689
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_title                3690 non-null   object 
 1   salary_estimate          3690 non-null   object 
 2   job_description          3690 non-null   object 
 3   rating                   3690 non-null   float64
 4   company_name             3690 non-null   object 
 5   location                 3690 non-null   object 
 6   headquarters             3690 non-null   object 
 7   size                     3690 non-null   object 
 8   founded                  3690 non-null   int64  
 9   type_of_ownership        3690 non-null   object 
 10  industry                 3690 non-null   object 
 11  sector                   3690 non-null   object 
 12  revenue                  3690 non-null   object 
 13  competitors              3690 non-null   object 
 14  easy_apply              

## Data Preprocessing

### Define stopwords to append to default stopwords

In [4]:
# append special stopwords in jobs listings to default and common stopwords
new_stopwords = STOPWORDS.union(set(['job', 'skill', 'experience', 'team', 'data', 'use', 'like', 'business', 'work', 'ability', 'let', 'example'
                'need', 'new', 'user', 'opportunity', 'candidate', 'provide', 'company', 'one', 'used', 'need', 
                'see', 'make', 'follow', 'going', 'will', 'want', 'well', 'find', 'give', 'change', 'look', 'first', 'using',
                'know', 'model', 'science', 'think', 'year', 'years', 'looking', 'machine', 'learning', 'job', 'description']))

### Method to preprocess data for jobs listings

In [5]:
# lemmatize = words change to noun
# stemmed = words reduced to root form
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='n')

# Convert a document into a list of tokens.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in new_stopwords:
            result.append(lemmatize(token))
    return result

In [6]:
preprocessed_jobs_listings = cleaned_jobs_listings_df['cleaned_job_description'].fillna('').astype(str).map(preprocess)

In [7]:
# implements the concept of a Dictionary – a mapping between words and their integer ids.
jobs_listings_dictionary = corpora.Dictionary(preprocessed_jobs_listings)

In [8]:
# Convert into the bag-of-words (BoW) format
jobs_listings_bow_corpus = [jobs_listings_dictionary.doc2bow(text) for text in preprocessed_jobs_listings]

In [9]:
# Convert into TF-IDF format
jobs_listings_tfidf = models.TfidfModel(jobs_listings_bow_corpus)
jobs_listings_tfidf_corpus = jobs_listings_tfidf[jobs_listings_bow_corpus]

## Common method to find dominant topics in the jobs description

In [10]:
def get_dominant_topics(model, corpus):
    dominant_topics_df = pd.DataFrame()
    for i, row in enumerate(model[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: # => dominant topic
                wp = model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                dominant_topics_df = dominant_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    dominant_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    dominant_topics_df["Text"] = cleaned_jobs_listings_df['cleaned_job_description']
    dominant_topics_df = dominant_topics_df.sort_values(by=['Perc_Contribution'], ascending=False)
    return dominant_topics_df.head(10)

## LSA Model

### Result using LSA model + Bag of words

In [11]:
jobs_listings_bow_lsamodel = LsiModel(jobs_listings_bow_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [12]:
jobs_listings_bow_lsamodel.print_topics(num_words=7)

[(0,
  '0.176*"development" + 0.166*"analysis" + 0.153*"skill" + 0.144*"product" + 0.141*"technology" + 0.139*"analytics" + 0.137*"information"'),
 (1,
  '0.253*"analytics" + -0.203*"information" + -0.200*"research" + -0.177*"employee" + -0.175*"laboratory" + 0.173*"product" + -0.163*"required"'),
 (2,
  '0.301*"research" + -0.272*"information" + -0.212*"application" + 0.192*"laboratory" + 0.186*"product" + 0.175*"clinical" + 0.164*"scientist"'),
 (3,
  '-0.229*"product" + -0.201*"status" + 0.198*"management" + -0.169*"people" + 0.158*"process" + -0.157*"gender" + 0.155*"project"'),
 (4,
  '0.388*"development" + -0.324*"analysis" + -0.284*"analytics" + 0.225*"technology" + -0.191*"statistical" + -0.141*"insight" + 0.140*"design"'),
 (5,
  '0.484*"research" + -0.326*"laboratory" + -0.214*"medical" + -0.194*"service" + 0.158*"analysis" + -0.154*"patient" + 0.153*"development"'),
 (6,
  '-0.427*"product" + 0.364*"research" + -0.222*"process" + 0.156*"solution" + 0.153*"technology" + 0.151

In [13]:
get_dominant_topics(jobs_listings_bow_lsamodel, jobs_listings_bow_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
1889,0.0,39.6850,"development, analysis, skill, product, technol...",job descriptionjob title data analystsalary 18...
591,0.0,39.4304,"development, analysis, skill, product, technol...",job descriptionoperation analytics manager tr...
891,0.0,32.8127,"development, analysis, skill, product, technol...",at west monroe our people are our businesswe p...
788,0.0,31.5336,"development, analysis, skill, product, technol...",address111 w monroe 115 s lasallejob family g...
1485,0.0,31.4038,"development, analysis, skill, product, technol...",descriptionthe lead of cybersecurity data prot...
770,0.0,28.3336,"development, analysis, skill, product, technol...",address111 w monroe 115 s lasallejob family g...
874,0.0,28.1692,"development, analysis, skill, product, technol...",please make sure to read the job posting in it...
471,0.0,27.2988,"development, analysis, skill, product, technol...",position descriptionposition title senior data...
97,0.0,27.1588,"development, analysis, skill, product, technol...",search by keywordsearch by locationclearsend m...
838,0.0,26.5936,"development, analysis, skill, product, technol...",please make sure to read the job posting in it...


### Result using LSA model + TF-IDF

In [14]:
jobs_listings_tfidf_lsamodel = LsiModel(jobs_listings_tfidf_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [15]:
jobs_listings_tfidf_lsamodel.print_topics(num_words=7)

[(0,
  '0.096*"analytics" + 0.087*"product" + 0.077*"research" + 0.077*"solution" + 0.076*"customer" + 0.073*"model" + 0.073*"client"'),
 (1,
  '-0.260*"laboratory" + -0.172*"clinical" + -0.158*"cell" + -0.115*"assay" + -0.114*"scientific" + 0.114*"aws" + -0.111*"biology"'),
 (2,
  '-0.181*"analyze" + -0.175*"locate" + -0.168*"printout" + -0.163*"databasesdata" + -0.149*"setsfilter" + -0.149*"qualityacquire" + -0.149*"disseminate"'),
 (3,
  '0.659*"tutor" + 0.442*"tutoring" + 0.163*"student" + 0.117*"choose" + 0.112*"session" + 0.087*"university" + -0.085*"cell"'),
 (4,
  '0.326*"tutor" + 0.218*"tutoring" + -0.161*"university" + -0.130*"clery" + -0.127*"security" + 0.120*"cell" + -0.117*"investigation"'),
 (5,
  '-0.171*"aws" + -0.159*"laboratory" + -0.145*"pipeline" + -0.144*"cell" + -0.136*"cloud" + -0.122*"spark" + -0.107*"etl"'),
 (6,
  '0.233*"cell" + 0.138*"biology" + -0.136*"laboratory" + -0.122*"hospital" + 0.117*"assay" + -0.109*"care" + -0.100*"reporting"'),
 (7,
  '0.252*"la

In [16]:
get_dominant_topics(jobs_listings_tfidf_lsamodel, jobs_listings_tfidf_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
3364,3.0,0.8857,"tutor, tutoring, student, choose, session, uni...",austin data science tutor jobs has students in...
1241,3.0,0.8818,"tutor, tutoring, student, choose, session, uni...",houston data science tutor jobs has students i...
1480,3.0,0.8812,"tutor, tutoring, student, choose, session, uni...",phoenix data science tutor jobs has students i...
3684,3.0,0.8800,"tutor, tutoring, student, choose, session, uni...",columbus data science tutor jobs has students ...
1981,3.0,0.8799,"tutor, tutoring, student, choose, session, uni...",san antonio data science tutor jobs has studen...
1244,3.0,0.8792,"tutor, tutoring, student, choose, session, uni...",west university place data science tutor jobs ...
1249,3.0,0.8786,"tutor, tutoring, student, choose, session, uni...",spring data science tutor jobs has students in...
3519,3.0,0.8783,"tutor, tutoring, student, choose, session, uni...",irving data science tutor jobs has students in...
3438,3.0,0.8773,"tutor, tutoring, student, choose, session, uni...",jacksonville data science tutor jobs has stude...
1510,3.0,0.8770,"tutor, tutoring, student, choose, session, uni...",tempe data science tutor jobs has students in ...


## LDA Model

### Result using LDA model + Bag of words

In [17]:
jobs_listings_bow_ldamodel = gensim.models.LdaMulticore(jobs_listings_bow_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [18]:
jobs_listings_bow_ldamodel.print_topics(num_words=7)

[(0,
  '0.009*"product" + 0.006*"analysis" + 0.005*"skill" + 0.005*"development" + 0.004*"solution" + 0.004*"technology" + 0.004*"working"'),
 (1,
  '0.007*"skill" + 0.005*"service" + 0.005*"project" + 0.005*"analysis" + 0.005*"including" + 0.004*"research" + 0.004*"analytics"'),
 (2,
  '0.007*"management" + 0.006*"analysis" + 0.006*"including" + 0.005*"product" + 0.005*"skill" + 0.005*"project" + 0.005*"support"'),
 (3,
  '0.007*"technology" + 0.005*"development" + 0.005*"analysis" + 0.005*"skill" + 0.004*"working" + 0.004*"scientist" + 0.004*"including"'),
 (4,
  '0.006*"product" + 0.006*"solution" + 0.005*"analysis" + 0.005*"engineering" + 0.005*"development" + 0.005*"technology" + 0.005*"analytics"'),
 (5,
  '0.006*"development" + 0.006*"analysis" + 0.005*"support" + 0.005*"skill" + 0.005*"project" + 0.004*"technology" + 0.004*"research"'),
 (6,
  '0.006*"development" + 0.006*"skill" + 0.005*"support" + 0.005*"analysis" + 0.005*"information" + 0.004*"process" + 0.004*"knowledge"'),

In [19]:
pyLDAvis.gensim_models.prepare(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.011118 -0.000409       1        1  21.388379
8      0.005695  0.009755       2        1  15.639191
5      0.000674  0.000949       3        1  10.583290
1     -0.000889 -0.003855       4        1   8.709956
9     -0.001086  0.001361       5        1   8.325809
3     -0.006283 -0.008171       6        1   8.270989
2     -0.011201  0.007763       7        1   8.084827
6     -0.005697 -0.005155       8        1   7.426357
0      0.008726 -0.005082       9        1   7.347695
7     -0.001058  0.002844      10        1   4.223507, topic_info=            Term         Freq        Total Category  logprob  loglift
132      product  4274.000000  4274.000000  Default  30.0000  30.0000
403   management  3208.000000  3208.000000  Default  29.0000  29.0000
297        skill  4455.000000  4455.000000  Default  28.0000  28.0000
50   development  4907.000000  4907.000000  Default  27.0000  27.0000
8       analysis  4770.000000  4770.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
305   technology   120.077445  4060.635117  Topic10  -5.7858  -0.3565
403   management   110.094073  3208.038540  Topic10  -5.8726  -0.2076
557       system   102.919317  2643.926397  Topic10  -5.9400  -0.0816
142     research   111.378057  3503.332701  Topic10  -5.8610  -0.2840
426      process   105.446424  3225.692183  Topic10  -5.9158  -0.2562

[852 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
12999      1  0.039802  accuracyadept
12999      2  0.398020  accuracyadept
12999      4  0.039802  accuracyadept
12999      5  0.039802  accuracyadept
12999      7  0.079604  accuracyadept
...      ...       ...            ...
18741      3  0.565273         企业岗位介绍
18744      3  0.566046        名bd实习若干
18747      3  0.566069  实习企业提供实习证明推荐信
18748      3  0.565634     家天使轮a轮融资预计
18752      3  0.565567          年起共累计

[2226 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 6, 2, 10, 4, 3, 7, 1, 8])

In [20]:
get_dominant_topics(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
97,7.0,0.9991,"development, analytics, related, analysis, pro...",search by keywordsearch by locationclearsend m...
96,7.0,0.9990,"development, analytics, related, analysis, pro...",search by keywordsearch by locationclearsend m...
941,1.0,0.9985,"skill, service, project, analysis, including, ...",levelrowlevelexperiencedjob locationchicago il...
17,4.0,0.9985,"product, solution, analysis, engineering, deve...",data scientistml engineerapply nowbecome part ...
522,1.0,0.9983,"skill, service, project, analysis, including, ...",we have aunique opportunity for someone to con...
2615,1.0,0.9982,"skill, service, project, analysis, including, ...",levelrowlevelmanagementjob locationdallas tx ...
1357,1.0,0.9982,"skill, service, project, analysis, including, ...",levelrowlevelexperiencedjob locationphoenix az...
3058,4.0,0.9982,"product, solution, analysis, engineering, deve...",named a best place to work in austin four yea...
1305,1.0,0.9982,"skill, service, project, analysis, including, ...",levelrowlevelexperiencedjob locationphoenix c...
311,8.0,0.9982,"development, analytics, tool, analysis, system...",senior data scientistnewark nj united statesjo...


### Result using LDA model + TF-IDF

In [21]:
jobs_listings_tfidf_ldamodel = gensim.models.LdaMulticore(jobs_listings_tfidf_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [22]:
jobs_listings_tfidf_ldamodel.print_topics(num_words=7)

[(0,
  '0.001*"research" + 0.001*"analytics" + 0.000*"analysis" + 0.000*"product" + 0.000*"statistical" + 0.000*"modeling" + 0.000*"client"'),
 (1,
  '0.001*"analytics" + 0.001*"product" + 0.001*"research" + 0.001*"client" + 0.001*"pipeline" + 0.000*"process" + 0.000*"tool"'),
 (2,
  '0.001*"analytics" + 0.001*"product" + 0.001*"database" + 0.001*"statistical" + 0.001*"model" + 0.001*"research" + 0.001*"solution"'),
 (3,
  '0.001*"analytics" + 0.001*"research" + 0.001*"customer" + 0.001*"analysis" + 0.001*"model" + 0.001*"statistical" + 0.001*"solution"'),
 (4,
  '0.001*"client" + 0.001*"product" + 0.001*"analytics" + 0.001*"research" + 0.001*"development" + 0.001*"cloud" + 0.001*"customer"'),
 (5,
  '0.001*"analytics" + 0.001*"product" + 0.001*"information" + 0.001*"research" + 0.001*"database" + 0.001*"statistical" + 0.001*"analysis"'),
 (6,
  '0.001*"bull" + 0.001*"product" + 0.001*"research" + 0.001*"clinical" + 0.001*"system" + 0.001*"analytics" + 0.001*"engineer"'),
 (7,
  '0.001

In [23]:
pyLDAvis.gensim_models.prepare(jobs_listings_tfidf_ldamodel, jobs_listings_tfidf_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.011448 -0.000151       1        1  15.865289
3      0.011985  0.000429       2        1  14.744196
6      0.010526 -0.011840       3        1  13.802486
4     -0.000255  0.014496       4        1  11.661224
7     -0.000143  0.010825       5        1  11.040607
5     -0.000591 -0.000581       6        1   8.413917
9     -0.004343 -0.002781       7        1   7.153835
1     -0.008009 -0.004873       8        1   6.785730
8     -0.006928 -0.000090       9        1   6.537409
0     -0.013689 -0.005434      10        1   3.995306, topic_info=              Term       Freq      Total Category  logprob  loglift
13281        tutor   7.000000   7.000000  Default  30.0000  30.0000
20615         bull   7.000000   7.000000  Default  29.0000  29.0000
13282     tutoring   5.000000   5.000000  Default  28.0000  28.0000
484       clinical  16.000000  16.000000  Default  27.0000  27.0000
2040         azure  11.000000  11.000000  Default  26.0000  26.0000
...            ...        ...        ...      ...      ...      ...
297          skill   0.580854  16.581974  Topic10  -7.8894  -0.1315
444       solution   0.623491  21.972385  Topic10  -7.8186  -0.3422
50     development   0.607922  20.589191  Topic10  -7.8439  -0.3024
655        support   0.579439  17.640450  Topic10  -7.8919  -0.1958
177           tool   0.578917  18.154844  Topic10  -7.8928  -0.2255

[797 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
199        1  0.145883        able
199        2  0.218825        able
199        3  0.145883        able
199        4  0.145883        able
199        5  0.072942        able
...      ...       ...         ...
197        8  0.059345     working
197        9  0.059345     working
9535       3  0.453816  yearsrsquo
4007       5  0.643788       young
19502      5  0.685819        zoom

[1253 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 7, 5, 8, 6, 10, 2, 9, 1])

In [24]:
get_dominant_topics(jobs_listings_tfidf_ldamodel, jobs_listings_tfidf_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
3359,6.0,0.9497,"bull, product, research, clinical, system, ana...",we are hiringdata engineer fulltime day aust...
3287,3.0,0.9487,"analytics, research, customer, analysis, model...",location pickle research campusjob posting tit...
3325,6.0,0.9486,"bull, product, research, clinical, system, ana...",we are hiringdata engineer fulltime day aust...
97,5.0,0.9485,"analytics, product, information, research, dat...",search by keywordsearch by locationclearsend m...
1050,4.0,0.9485,"client, product, analytics, research, developm...",who is is a leading transformative it authori...
3261,6.0,0.9483,"bull, product, research, clinical, system, ana...",why join technologies leverages technology to...
3362,3.0,0.9479,"analytics, research, customer, analysis, model...",job posting titledevopsbuild and release engin...
1571,2.0,0.9476,"analytics, product, database, statistical, mod...",site name usa pennsylvania upper providence ...
3276,3.0,0.9476,"analytics, research, customer, analysis, model...",location pickle research campusjob posting tit...
96,5.0,0.9472,"analytics, product, information, research, dat...",search by keywordsearch by locationclearsend m...
